In [3]:
import json
import pandas as pd


file_path_qa = "/mnt/data/wangshu/hcarag/mintaka/data/mintaka_test.json"
qa_df_raw = pd.read_json(file_path_qa)
qa_df_raw.head(2)


,id,question,translations,questionEntity,answer,category,complexityType
0,fae46b21,What man was a famous American author and also...,{'ar': 'مَن الرجل الذي كان مؤلفًا أمريكيًا مشه...,"[{'name': 'Q1497', 'entityType': 'entity', 'la...","{'answerType': 'entity', 'answer': [{'name': '...",history,intersection
1,bc8713cc,How many Academy Awards has Jake Gyllenhaal be...,{'ar': 'كم عدد جوائز الأوسكار التي ترشح لها ال...,"[{'name': 'Q133313', 'entityType': 'entity', '...","{'answerType': 'numerical', 'answer': [1], 'me...",movies,count


In [25]:
qa_df = qa_df_raw.copy()

sel_comples = [
    "superlative",
    "ordinal",
    "multihop",
    "intersection",
    "difference",
    "generic",
]
qa_df = qa_df[qa_df["complexityType"].isin(sel_comples)]
qa_df = qa_df[["question", "answer", "complexityType"]]

qa_df['ansType'] = qa_df['answer'].apply(lambda x: x['answerType'])
an_type_sel = ['entity', 'date', 'numerical']
qa_df = qa_df[qa_df['ansType'].isin(an_type_sel)]

print(qa_df["ansType"].value_counts())
print(qa_df.shape)
qa_df.head(2)

ansType
entity       2274
date          265
numerical     254
Name: count, dtype: int64
(2793, 4)


,question,answer,complexityType,ansType
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",intersection,entity
5,Who performed at the Super Bowl XXIII halftime...,"{'answerType': 'entity', 'answer': None, 'ment...",generic,entity


In [26]:
def get_ans_list(row):
    mention_str = row["answer_raw"].get("mention", "").strip()
    mention_str = mention_str.replace("and", ",")
    mention_list = mention_str.split(",")
    mention_list = [mention.strip() for mention in mention_list]
    if row["ansType"] == "entity":
        ans_list = row["answer_raw"].get("answer", None)
        if ans_list is None:
            label_list = mention_list
        else:
            entity_list = []
            for ans in ans_list:
                entity_list.append(ans["label"]["en"])
            label_list = mention_list + entity_list
    else:
        label_list =[str(row["answer_raw"]["answer"][0])] + mention_list
    
    label_list = [label.lower() for label in label_list]
    label_list = list(set(label_list))
    label_str = "|".join(label_list)
    return label_str


qa_df.rename(columns={"answer": "answer_raw"}, inplace=True)
qa_df["label"] = qa_df.apply(get_ans_list, axis=1)

qa_df.head(2)

,question,answer_raw,complexityType,ansType,label
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",intersection,entity,mark twain
5,Who performed at the Super Bowl XXIII halftime...,"{'answerType': 'entity', 'answer': None, 'ment...",generic,entity,elvis presto


In [27]:
import sys
import os

# F# 添加 src 文件夹到 sys.path
src_path = "/home/wangshu/rag/hier_graph_rag"
sys.path.append(os.path.abspath(src_path))
from src.utils import relation_embedding

class Args:
    def __init__(self):
        self.embedding_local = False
        self.embedding_api_key = "ollama"
        self.embedding_api_base = "http://localhost:5000/forward"
        self.embedding_model = "nomic-embed-text"


args = Args()
qa_df, _ = relation_embedding(
    qa_df,
    args=args,
    e_colname="question",
    embed_colname="embedding",
    num_workers=48,
    embed_in_ori=True,
)

qa_df.head(2)

local is False
the number of unique question is 2793


Computing embeddings: 100%|██████████| 2793/2793 [00:41<00:00, 67.05it/s] 


,question,answer_raw,complexityType,ansType,label,embedding
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",intersection,entity,mark twain,"[-0.022767667, 0.026325747, -0.21478522, -0.03..."
5,Who performed at the Super Bowl XXIII halftime...,"{'answerType': 'entity', 'answer': None, 'ment...",generic,entity,elvis presto,"[-0.046611417, 0.038441464, -0.19116583, 0.003..."


In [28]:
save_path_qa = "/mnt/data/wangshu/hcarag/mintaka/QA/mintaka_test_qa.json"
qa_df.to_json(save_path_qa, orient="records", lines=True)

qa_df.head(2)

,question,answer_raw,complexityType,ansType,label,embedding
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",intersection,entity,mark twain,"[-0.022767667, 0.026325747, -0.21478522, -0.03..."
5,Who performed at the Super Bowl XXIII halftime...,"{'answerType': 'entity', 'answer': None, 'ment...",generic,entity,elvis presto,"[-0.046611417, 0.038441464, -0.19116583, 0.003..."
